Setup random reach task

In [1]:
from ctd.task_modeling.task_env.task_env import RandomTarget
from motornet.effector import RigidTendonArm26
from motornet.muscle import MujocoHillMuscle
# Create the analysis object:
rt_task_env = RandomTarget(effector = RigidTendonArm26(muscle = MujocoHillMuscle()))

Train LSTM model to perform this task

In [2]:
from ctd.task_modeling.datamodule.task_datamodule import TaskDataModule
from ctd.task_modeling.task_wrapper.task_wrapper import TaskTrainedWrapper
from pytorch_lightning import Trainer

from Models.LSTM import LSTM_cell

# Initialize the task environment 
task_env = rt_task_env

# Initialize the model 
in_size = task_env.observation_space.shape[0] + task_env.context_inputs.shape[0]
out_size = task_env.action_space.shape[0]
lstm_model = LSTM_cell(output_size=2, input_size= in_size, latent_size=128)

# Setup the task in the datamodule
task_dm = TaskDataModule(task_env,n_samples=1000 ,batch_size=256)

# Setup the environment and model in the task wrapper
task_wrapper = TaskTrainedWrapper(learning_rate=1e-3, weight_decay=1e-8)
task_wrapper.set_environment(task_env)
task_wrapper.set_model(lstm_model)

# Define trainer
trainer = Trainer(accelerator = 'cpu', max_epochs=500, enable_progress_bar=False,)

# Train the model
trainer.fit(task_wrapper, task_dm)

/home/juanpbotero/anaconda3/envs/NMA-NeuroAI/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'data_env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['data_env'])`.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/juanpbotero/anaconda3/envs/NMA-NeuroAI/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name     | Type         | Params
------------------------------------------
0 | task_env | RandomTarget | 84    
1 | model    | LSTM_cell    | 75.5 K
------------------------------------------
75.5 K    Trainable params
84        Non-trainable params
75.6 K    Total params
0.302     Total estimated model params size (MB)


ValueError: too many values to unpack (expected 2)